# 관광객 추정 버스 이용자 추출 알고리즘
* input: 버스 이용 데이터
* ouput: [user_id, first_usage_date, last_usage_date, tourist_flag]

In [1]:
import os
import datetime
import pandas as pd
from tqdm import tqdm
from pyarrow import csv

# 사용자 정의
import bus

In [2]:
# 관광객 연속 체류 기간 (default: 15)
day = 15

geton_station_name = "geton_stataion_name"

### 1. Def

#### 1) 데이터 전처리
* 날짜 데이터 int -> datetime 변환

In [3]:
def data_preprocessing(usage_df):
    # datetime64로 형 변환 # M[base_date] = pd.to_datetime(M[base_date], format='%Y%m%d')
    datetime_cols = ["geton_datetime", "getoff_datetime"]    
    for col in tqdm(datetime_cols):
        usage_df[col] = pd.to_datetime(usage_df[col], format='%Y%m%d%H%M%S')
    return usage_df

#### 2) 전체 유저 이용데이터 분석
* 전체 사용자 리스트 추출
* 사용기간 분석

In [4]:
def analyze_total_user(usage_df):
    user_id_df = usage_df[["user_id"]].drop_duplicates()

    geton_grouped_df = usage_df[["user_id", "geton_datetime"]].groupby(["user_id"])
    getoff_grouped_df = usage_df[["user_id", "getoff_datetime"]].groupby(["user_id"])

    first_use_date_df = geton_grouped_df.min().rename(columns = {"geton_datetime": "first_usage_datetime"})

    geton_last_use_date_df = geton_grouped_df.max().rename(columns = {"geton_datetime": "last_usage_datetime"})
    getoff_last_use_date_df = getoff_grouped_df.max().rename(columns = {"geton_datetime": "last_usage_datetime"})
    last_use_date_df = geton_last_use_date_df

    user_infor_df = pd.merge(user_id_df, first_use_date_df, on = "user_id")
    user_infor_df = pd.merge(user_infor_df, last_use_date_df, on = "user_id")
    user_infor_df["tour_period"] = user_infor_df["last_usage_datetime"] - user_infor_df["first_usage_datetime"]
    user_infor_df["tour_period"]
    
    return user_infor_df

#### 3) 추출1 - 사용기간 고려
관광객으로 유추 가능한 사용자 1차 추출.
1. {user_id, base_date} 쌍을 집합(중복 제거) user_id_and_base_date에 담는다.
    이때, 중복 제거는 {'a1b1c1', '20190101'}, {'a1b1c1', '20190101'} ... 
    => 2회 이상 중복되는 것을 1개만 유지하고 모두 삭제처리한다는 의미.
    * 중복이 생긴 이유는 2019년 1월 1일 a1b1c1이 버스를 3번 탔으면, 3번만큼 튜플이 생기는 것이 당연. 
2. 이후 집합 user_id_and_base_date에 대하여 groupby(user_id).count() 연산을 수행하면,
    각 user가 며칠 버스를 이용하였는지 알 수 있다. (단, 연속적인 며칠 X.) 
    경우1 (1월 1일, 2월 3일, ..., 12월 1일 => 10일): 불연속적인 10일
    경우2 (1월 1일, 2일, 3일, ..., 10일 => 10일): 연속적인 10일
참고로, 2차 추출 단계에서 경우2에 해당하는 튜플만 다시 추출할 것이다.

In [5]:
def extract_user1(usage_df):
    user_id_and_base_date = usage_df[["user_id", "base_date"]].drop_duplicates()
    date_cnt = user_id_and_base_date.groupby(by=["user_id"], as_index=False).count()
    U = list(date_cnt[date_cnt["base_date"] < day]["user_id"])
    return U

#### 4) 추출2 - 사용기간 고려
관광객으로 유추 가능한 사용자 2차 추출.
1. 1차 추출에 의해 식별된 user_id의 인스턴스들을 모아 M2에 저장.
2. 이후 user_id별로 base_date의 max와 min을 추출
3. user_id별로 max - min이 14를 초과하는 경우 삭제 처리
   (연속적으로 며칠 이용했는지 판별 -> 14일 이하인 것만 추출.)

In [6]:
def extract_user2(usage_df, U):
    M2 = usage_df.query('{} in {}'.format("user_id", U))[["user_id", "base_date"]].groupby(by=["user_id"], as_index=False)
    M2_left = M2.max()
    M2_right = M2.min()
    suffixes = ['_max', '_min']

    M2_2 = pd.merge(M2_left, M2_right, on="user_id", how='inner', suffixes=suffixes)
    for postfix in suffixes:
        M2_2["base_date" + postfix] = pd.to_datetime(M2_2["base_date" + postfix], format='%Y%m%d')
    M2_2['diff'] = M2_2["base_date" + suffixes[0]] - M2_2["base_date" + suffixes[1]]

    U2 = list(M2_2[M2_2['diff'] < '%d days' % day]["user_id"].unique()) 
    return U2

#### 5) 추출3 - 방문 정류장 고려
관광객으로 유추 가능한 사용자 3차 추출.
1. 승/하차 정류장 중 '제주국제공항'이 존재하는가?
2. 승/하차 정류장 중 '제주연안여객터미널', '국제여객선터미널'등이 존재하는가?
3. 승/하차 정류장 중 '호텔/여관/펜션' 등이 존재하는가?

In [7]:
def extract_user3(user_df, U2):
    M3 = user_df.query('{} in {}'.format("user_id", U2))

    query_set = []
    tourist_area_A = ['제주국제공항', '제주연안여객터미널', '국제여객선터미널',
                      '호텔', '여관', '펜션', '민박', '관광', '해수욕장', '공원', '파크', '랜드', '휴양림', '박물관', '폭포', '미술관', '오름']
    tourist_area_B = ['제주국제공항', '제주연안여객터미널', '국제여객선터미널', '호텔', '여관', '펜션', '민박', '관광']
    
    for col in [geton_station_name, "getoff_station_name"]:
        for col2 in tourist_area_B:
            query = '{0}.str.contains("{1}") '.format(col, col2)
            query_set.append(query)
    query = ' or '.join(query_set)
    M3 = M3.query(query, engine='python')
    U3 = list(M3["user_id"].unique())
    return U3

### 2. main

In [8]:
start_date = datetime.datetime(2019, 6, 1)
end_date = datetime.datetime(2019, 8, 29)

input_path_list = bus.make_input_path(start_date, end_date)

usage_df = bus.load_total_usage_data(input_path_list)
usage_df = data_preprocessing(usage_df)
print('추출 전 user_id의 개수', len(usage_df["user_id"])) 

user_infor_df = analyze_total_user(usage_df)

U = extract_user1(usage_df)
print('1차 추출 결과 user_id의 개수:', len(U))

U2 = extract_user2(usage_df, U)
print('2차 추출 결과 user_id의 개수:', len(U2))

U3 = extract_user3(usage_df, U2)
print('3차 추출 결과 user_id의 개수:', len(U3))
#유의사항: 실제 사용자(관광객) 수는 user_id의 개수보다 크거나 같을 것. (사용자 수는 user_count로 파악할 것.)

user_infor_df["tourist"] = user_infor_df["user_id"].isin(U3).apply(lambda x : int(x))
user_infor_df.to_csv("user_list.csv", encoding="utf-8", index=False)
print("저장완료")

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:50<00:00, 25.14s/it]


추출 전 user_id의 개수 12291789
1차 추출 결과 user_id의 개수: 605906
2차 추출 결과 user_id의 개수: 435662
3차 추출 결과 user_id의 개수: 227624
저장완료


### 3. 결과 출력 

In [9]:
user_infor_df

,user_id,first_usage_datetime,last_usage_datetime,tour_period,tourist
0,f6f372cf8c6732eafc2a82b4f9d7a08bb3b493213ea4ef...,2019-06-01 15:02:25,2019-06-09 20:47:31,8 days 05:45:06,1
1,3f6cfe2429f6e8d62cddc39222993405789b7b9ea2a592...,2019-06-01 15:02:29,2019-08-28 18:45:54,88 days 03:43:25,0
2,107cd874732b48a35df957bf653751154b0234e5573ba5...,2019-06-01 07:20:13,2019-08-28 16:09:42,88 days 08:49:29,0
3,27b16001fafd812b1e5c474cbc60455a9286ad27ecf8e5...,2019-06-01 07:45:59,2019-08-28 18:32:47,88 days 10:46:48,0
4,89c9a1c6e5915df4c192acc54067ee9a743f44eb559f3b...,2019-06-01 17:42:06,2019-08-28 19:38:13,88 days 01:56:07,0
...,...,...,...,...,...
726317,1c0b23153f52f11215e7d1f4e45dcecfa6d669676c4a9f...,2019-08-28 18:30:20,2019-08-28 18:30:20,0 days 00:00:00,0
726318,8235b6d43ba2e06d89fe94fda0dfdb7b4fe678ac58eaf5...,2019-08-28 18:54:37,2019-08-28 18:54:37,0 days 00:00:00,0
726319,f6f259fe96fd2e245be452a154c7c43ac747bc0f5d5ad1...,2019-08-29 00:16:52,2019-08-29 00:16:52,0 days 00:00:00,0
726320,670048e0bb0ed87661ef4e33fe476d74e4616c137bcdb0...,2019-08-29 00:17:12,2019-08-29 00:17:12,0 days 00:00:00,0


# 추출된 임의의 관광객 이동 패턴 출력 예시

In [10]:
usage_df.query('{} == "{}"'.format("user_id", 
                           '250557c8a203138628194b0ed3dfe9c711c3d21d09d984916c4c9546134f22be'))

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date
6290732,250557c8a203138628194b0ed3dfe9c711c3d21d09d984...,20190712,23100000,325-11(한라수목원~신사동(종점)),325-11,2019-07-12 08:29:40,638,제주국제공항(구제주방면),126.49345,33.50661,2019-07-12 09:17:45,198.0,함덕환승정류장(함덕해수욕장),126.66877,33.54101,일반,1,20190712
6506856,250557c8a203138628194b0ed3dfe9c711c3d21d09d984...,20190714,23110000,325-12(신사동(기점)~한라수목원),325-12,2019-07-14 09:00:38,394,함덕리(3구),126.66660,33.54171,2019-07-14 09:48:32,1355.0,제주국제공항(신제주방면),126.49275,33.50610,일반,1,20190714


In [11]:
usage_df.query('{} == "{}"'.format("user_id", 
                           '055373583405cd9f943d40b199d52570332f3ac91e2f85b726e67da48284d2a8'))

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date
9925431,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190807,28500000,201-13(제주버스터미널(가상정류소)~서귀포버스터미널),201-13,2019-08-07 17:03:30,1565,시외버스출발지(가상정류소),126.51478,33.49946,NaT,NaN,NaN,NaN,NaN,일반,1,20190808
10081791,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190808,21520000,182-1(제주버스터미널~제주국제공항),182-1,2019-08-08 10:26:54,6100848,제주시외버스터미널,126.51480,33.49955,2019-08-08 11:54:05,1784.0,중앙로터리(동),126.56135,33.25297,일반,1,20190808
10086550,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190808,4280000,600-2(서귀포칼호텔~제주국제공항),600-2,2019-08-08 19:44:18,1685,플레이케이팝박물관,126.41415,33.25007,2019-08-08 20:41:52,2972.0,제주국제공항(종점),126.49319,33.50571,일반,1,20190808
10088404,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190808,23140000,331-1(한라수목원~삼양종점),331-1,2019-08-08 20:56:32,638,제주국제공항(구제주방면),126.49345,33.50661,2019-08-08 21:07:17,148.0,남서광마을입구,126.52126,33.49968,일반,1,20190808
10226391,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190808,4280000,600-2(서귀포칼호텔~제주국제공항),600-2,2019-08-08 14:39:54,2104,뉴경남호텔,126.55825,33.24671,NaT,NaN,NaN,NaN,NaN,일반,1,20190810
10230796,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190809,28490000,201-12(서귀포버스터미널(가상정류소)~제주버스터미널(종점)),201-12,2019-08-09 20:20:42,192,함덕환승정류장(함덕해수욕장),126.66914,33.54101,2019-08-09 21:03:30,2969.0,제주버스터미널(종점),126.51632,33.49919,일반,1,20190809
10258816,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190809,28480000,201-11(제주버스터미널(가상정류소)~서귀포버스터미널),201-11,2019-08-09 11:58:44,1565,시외버스출발지(가상정류소),126.51478,33.49946,2019-08-09 12:32:13,224.0,삼양2동,126.58042,33.52118,일반,1,20190809
10384202,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190809,28540000,201-17(제주버스터미널(가상정류소)~서귀포버스터미널),201-17,2019-08-09 16:30:16,221,삼양초등학교,126.58611,33.52112,2019-08-09 16:48:18,198.0,함덕환승정류장(함덕해수욕장),126.66877,33.54101,일반,1,20190811
10388515,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190810,28580000,201-21(제주버스터미널(가상정류소)~서귀포버스터미널),201-21,2019-08-10 11:54:25,148,남서광마을입구,126.52126,33.49968,2019-08-10 12:55:15,1222.0,만장굴입구,126.77730,33.54925,일반,1,20190812
10435457,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190810,28490000,201-12(서귀포버스터미널(가상정류소)~제주버스터미널(종점)),201-12,2019-08-10 17:25:29,859,동성동,126.75453,33.55577,NaT,NaN,NaN,NaN,NaN,일반,1,20190810


In [12]:
usage_df.query('{} == "{}"'.format("user_id", 
                           'f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10494887ebb5e470023b')).sort_values(by="geton_datetime")

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date
12234443,f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10...,20190828,23620000,365-22(제주한라대학교~제주대학교),365-22,2019-08-28 10:20:02,638,제주국제공항(구제주방면),126.49345,33.50661,2019-08-28 10:29:32,323.0,서문시장,126.51845,33.51203,일반,1,20190828
12241185,f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10...,20190828,30240000,447-2(제주대학교~도평초등학교),447-2,2019-08-28 11:21:15,325,관덕정,126.52088,33.51322,2019-08-28 11:32:41,417.0,어영마을,126.48947,33.51901,일반,1,20190828
12234855,f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10...,20190828,24220000,445-5(도평동~도평초등학교),445-5,2019-08-28 13:53:17,416,어영마을,126.49008,33.51901,2019-08-28 14:08:36,324.0,관덕정,126.52096,33.51293,일반,1,20190828
12245161,f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10...,20190828,24310000,466-1(축산마을~축산진흥원입구),466-1,2019-08-28 14:13:08,325,관덕정,126.52088,33.51322,2019-08-28 14:47:46,480.0,연동대림1차아파트,126.48975,33.48028,일반,1,20190828
12227151,f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10...,20190828,23180000,332-1(한라수목원~삼양종점),332-1,2019-08-28 16:16:05,465,제주고등학교,126.48392,33.47433,2019-08-28 16:58:22,282.0,중앙로(동문시장),126.52457,33.51168,일반,1,20190828


In [13]:
usage_df.query('{} == "{}"'.format("user_id", 
                           '1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb8e8822b47397cf3e11')).sort_values(by="geton_datetime")

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date
10996741,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190814,4270000,600-1(제주국제공항(600번)~서귀포칼호텔(종점)),600-1,2019-08-14 15:42:25,2966,롯데시티호텔(600번),126.48856,33.49002,NaT,NaN,NaN,NaN,NaN,일반,1,20190814
11127724,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190815,31630000,521-2(효돈중학교~제주국제컨벤션센터 중문대포해안주상절리대),521-2,2019-08-15 09:51:32,1963,솔동산입구,126.56368,33.24474,NaT,NaN,NaN,NaN,NaN,일반,1,20190815
11127849,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190815,25050000,520-2(효돈중학교~제주국제컨벤션센터 중문대포해안주상절리대),520-2,2019-08-15 10:38:44,2700,약천사,126.45034,33.24300,NaT,NaN,NaN,NaN,NaN,일반,1,20190815
11099077,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190815,25050000,520-2(효돈중학교~제주국제컨벤션센터 중문대포해안주상절리대),520-2,2019-08-15 11:34:30,3585,동물개동산앞,126.43120,33.24012,2019-08-15 11:43:10,1951.0,중문관광단지,126.41271,33.25220,일반,1,20190815
11088973,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190815,28720000,202-16(구 터미널~제주버스터미널),202-16,2019-08-15 14:39:43,2137,천제연폭포,126.41898,33.25395,2019-08-15 15:03:47,2465.0,산방산,126.31245,33.23623,일반,1,20190815
11109960,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190815,28710000,202-15(제주버스터미널(가상정류소)~서귀포등기소),202-15,2019-08-15 15:42:44,2464,산방산,126.31248,33.23612,2019-08-15 16:19:08,2020.0,평생학습관,126.55807,33.24950,일반,1,20190815
11201696,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190816,25040000,520-1(제주국제컨벤션센터~효돈중학교),520-1,2019-08-16 08:00:49,1962,솔동산입구,126.56369,33.24461,2019-08-16 08:03:07,1802.0,주공34단지,126.57187,33.25010,일반,1,20190816
11217063,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190816,28490000,201-12(서귀포버스터미널(가상정류소)~제주버스터미널(종점)),201-12,2019-08-16 08:04:10,1802,주공34단지,126.57187,33.25010,2019-08-16 09:27:22,2354.0,성산항입구,126.92917,33.46849,일반,1,20190816
11191733,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190816,28500000,201-13(제주버스터미널(가상정류소)~서귀포버스터미널),201-13,2019-08-16 15:28:14,2353,성산항입구,126.92955,33.46820,2019-08-16 16:55:35,2088.0,동문로터리,126.56835,33.24868,일반,1,20190816
11314430,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190819,4280000,600-2(서귀포칼호텔~제주국제공항),600-2,2019-08-19 10:48:05,2104,뉴경남호텔,126.55825,33.24671,NaT,NaN,NaN,NaN,NaN,일반,1,20190821


In [14]:
usage_df.query('{} == "{}"'.format("user_id", 
                           '68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4fa4afe6255d7b33d58')).sort_values(by="geton_datetime")

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date
6953192,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190716,30040000,365-29(제주한라대학교~제주국제대학교),365-29,2019-07-16 08:28:46,638,제주국제공항(구제주방면),126.49345,33.50661,2019-07-16 09:02:59,132.0,아라초등학교,126.54428,33.47653,일반,1,20190717
6878889,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190716,22520000,281-1(제주버스터미널(가상정류소)~서귀포버스터미널),281-1,2019-07-16 09:12:20,132,아라초등학교,126.54428,33.47653,2019-07-16 09:22:03,135.0,산천단,126.55539,33.44786,일반,1,20190716
6872828,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190716,22200000,212-1(제주버스터미널(가상정류소)~성산항),212-1,2019-07-16 11:38:58,135,산천단,126.55539,33.44786,2019-07-16 11:49:37,4064.0,명도암입구삼거리,126.64044,33.42757,일반,1,20190716
6886748,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190716,32500000,232-4(중앙로터리~제주버스터미널(종점)),232-4,2019-07-16 15:46:03,2719,붉은오름,126.68525,33.39522,2019-07-16 15:48:14,1262.0,제주목장,126.67799,33.41741,일반,1,20190716
6905519,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190716,32480000,232-2(중앙로터리~제주버스터미널(종점)),232-2,2019-07-16 17:23:19,1262,제주목장,126.67799,33.41741,2019-07-16 17:46:14,136.0,산천단한국폴리텍대학,126.55558,33.44799,일반,1,20190716
7084848,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190717,22530000,281-2(서귀포버스터미널(가상정류소)~제주버스터미널(종점)),281-2,2019-07-17 06:40:33,138,사회복지법인춘강,126.55164,33.45264,2019-07-17 07:00:35,2969.0,제주버스터미널(종점),126.51632,33.49919,일반,1,20190717
6985223,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190717,22340000,240-1(제주버스터미널(가상정류소)~제주국제컨벤션센터),240-1,2019-07-17 07:22:12,1564,제주시외버스터미널,126.51479,33.49946,2019-07-17 08:32:15,2636.0,영실매표소,126.48022,33.34025,일반,1,20190718
7086344,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190717,22350000,240-2(제주국제컨벤션센터 중문대포해안주상절리대~제주버스터미널(종점)),240-2,2019-07-17 14:38:02,2636,영실매표소,126.48022,33.34025,2019-07-17 15:39:52,2970.0,제주시외버스터미널(종점),126.51414,33.49993,일반,1,20190717
7090186,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190717,23490000,355-21(수산리사무소앞~제주대학교),355-21,2019-07-17 15:56:29,149,제주버스터미널,126.51486,33.49993,2019-07-17 16:08:13,3276.0,제주중앙여자고등학교(아라방면),126.53864,33.48941,일반,1,20190717
7135520,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190717,22520000,281-1(제주버스터미널(가상정류소)~서귀포버스터미널),281-1,2019-07-17 16:53:32,3276,제주중앙여자고등학교(아라방면),126.53864,33.48941,NaT,NaN,NaN,NaN,NaN,일반,1,20190717


In [15]:
usage_df.query('{} == "{}"'.format("user_id", 
                           'a6583c22116492a72c059e41272ef7aacdc29f6e9138ede6cf5142c26d780fb4'))

,user_id,base_date,route_id,route_name,route_no,geton_datetime,geton_station_id,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_id,getoff_station_name,getoff_station_longitude,getoff_station_latitude,user_type,user_count,input_date
2970326,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190620,23710000,316-2(삼양종점~번대동),316-2,2019-06-20 07:08:31,1355,제주국제공항(신제주방면),126.49275,33.50610,NaT,NaN,NaN,NaN,NaN,청소년,1,20190620
3090549,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190621,22190000,211-2(성산항~제주버스터미널(종점)),211-2,2019-06-21 18:40:26,2355,성산항,126.92994,33.47043,2019-06-21 19:54:01,2969.0,제주버스터미널(종점),126.51632,33.49919,청소년,1,20190621
3126690,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190621,21180000,"111-1(제주국제공항(표선,성산,남원)~성산항)",111-1,2019-06-21 11:19:06,149,제주버스터미널,126.51486,33.49993,2019-06-21 12:24:21,2221.0,성산항,126.93008,33.47032,청소년,1,20190621
3191571,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190622,29210000,282-5(제주버스터미널(가상정류소)~서귀포시 구 버스터미널),282-5,2019-06-22 11:27:06,173,명신마을,126.50303,33.49962,2019-06-22 12:23:41,2052.0,중문환승정류장(중문우체국),126.42380,33.25180,청소년,1,20190622
3328058,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190623,21480000,"181-1(제주국제공항(일주동로,516도로)~제주버스터미널(종점))",181-1,2019-06-23 13:37:26,6103001,제주월드컵경기장,126.50862,33.24948,2019-06-23 14:35:38,320.0,제주도청신제주로터리,126.49678,33.49143,청소년,1,20190623
3357893,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190623,23620000,365-22(제주한라대학교~제주대학교),365-22,2019-06-23 14:50:05,320,제주도청신제주로터리,126.49678,33.49143,2019-06-23 15:08:05,324.0,관덕정,126.52096,33.51293,청소년,1,20190623
3424581,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190624,32240000,752-3(모슬포 남항 여객선 터미널(운진항)~모슬포 남항 여객선 터미널(운진항)),752-3,2019-06-24 12:36:27,1639,상창보건진료소,126.36294,33.27464,2019-06-24 12:40:13,2180.0,상천리사무소,126.37999,33.30723,청소년,1,20190624
3431038,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190624,22350000,240-2(제주국제컨벤션센터 중문대포해안주상절리대~제주버스터미널(종점)),240-2,2019-06-24 16:12:23,1660,제주국제컨벤션센터중문대포해안주상절리대,126.42472,33.24309,2019-06-24 16:54:46,1249.0,어리목,126.48531,33.39035,청소년,1,20190624
3433113,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190624,21520000,182-1(제주버스터미널~제주국제공항),182-1,2019-06-24 11:28:03,321,제주도청신제주로터리,126.49647,33.49110,2019-06-24 12:03:24,1640.0,상창보건진료소,126.36280,33.27458,청소년,1,20190624
3456616,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190624,4280000,600-2(서귀포칼호텔~제주국제공항),600-2,2019-06-24 15:38:50,2972,제주국제공항(종점),126.49356,33.50572,2019-06-24 16:04:33,2825.0,중문관광단지여미지식물원입구,126.41264,33.25170,일반,1,20190625
